In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv("data/men_shoes.csv", low_memory=False)

In [6]:
df.columns  

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats,model=DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model,X,y,scoring= 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [8]:
df['brand_cat'] = df['brand'].factorize()[0]
print("DecisionTreeRegressor model outcome: ", run_model(['brand_cat']))
model = RandomForestRegressor(max_depth=5,
                              n_estimators=100,
                              random_state=0)
print("RandomForestRegressor model outcome: ", run_model(['brand_cat'],model))

DecisionTreeRegressor model outcome:  (-58.38655694633361, 4.223555478221712)
RandomForestRegressor model outcome:  (-57.47223572384038, 4.328288468270897)


In [9]:
df['brand_cat2'] = df['brand'].map(lambda x:str(x).lower()).factorize()[0]
print("DecisionTreeRegressor model outcome: ", run_model(['brand_cat2']))
print("RandomForestRegressor model outcome: ", run_model(['brand_cat2'],model))

DecisionTreeRegressor model outcome:  (-58.133398968282776, 4.206122611474276)
RandomForestRegressor model outcome:  (-57.31783843165656, 4.181246596160967)


In [10]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [11]:
# widzimy, ze features to slownik zapisany jako str, musimy wrocic do formy slownika aby uzywac tej kolumny
# ponizej funkcja literal_eval ktora nam ten process ulatwi

str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [12]:
# chcemy je miec w takiej formie jak ponizej
{ 
    'Gender': 'Men',
    'Shoe Size': 'M',
    'Shoe Category': "Men's shoes",
    'Color': 'Multicolor',
    'Manufacturer Part Number': '8190-W-NAVY-7.5',
    'Brand': 'Josmo'
 }

{'Brand': 'Josmo',
 'Color': 'Multicolor',
 'Gender': 'Men',
 'Manufacturer Part Number': '8190-W-NAVY-7.5',
 'Shoe Category': "Men's shoes",
 'Shoe Size': 'M'}

In [13]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    # theat's how item look right now
    # {'key': 'Gender', 'value': ['Men']}
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict


df['features_parsed'] = df['features'].map(parse_features)
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [15]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [17]:
def get_name_feat(key):
  return "feat_" + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [18]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_battery type', 'feat_age end', 'feat_gender', 'feat_year',
       'feat_clothing size', 'feat_fabric care instructions',
       'feat_productgroup', 'feat_country/region of manufacture',
       'feat_number of compartments', 'feat_technician'],
      dtype='object', length=527)

In [0]:
keys_stats = {}
for key in keys:
  keys_stats[key] = df [ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [20]:
{k:v for k,v in keys_stats.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_brand'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]

In [22]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df [ df.brand == df.feat_brand].shape

(8846, 1003)

In [23]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat2'],model)

(-57.356966891333705, 4.227110789126767)

In [24]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'brand_cat2',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_certifications and listings',
 'feat_recommended location',
 'feat_shoe category',
 'feat_location - country',
 'feat_fabrication',
 'feat_catalog',
 'feat_location - city/state',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_made in_cat',
 'feat_chest_cat',
 'feat_general warranty_cat',
 'feat_suitable for_cat',
 'feat_mechanic_cat',
 'feat_case back_cat',
 'feat_rise_cat',
 'feat_diameter_cat',
 'feat_gps_cat',
 'feat_weather-resistant_cat',
 'feat_polarised_cat',
 'feat_wheel type_cat',
 'feat_free item_cat',
 'feat_name_cat',
 'feat_frame type_cat',
 'feat_toe type_cat',
 'feat_width_cat',
 'feat_polarized?_cat',
 'feat_clothing category_cat',
 'feat_bridge width_cat',
 'feat_compatible devices_cat',
 'feat_case type_cat',
 'feat_item type_cat',
 'feat_bezel_ca

In [0]:
feats = [
         'brand_cat2',
         'feat_brand_cat',
         'feat_gender_cat',
         'feat_material_cat']
feats_catched = [
                 'feat_weight_cat',
                 'feat_metal type_cat',
                 'feat_movement_cat',
                 'feat_adjustable_cat',
                 'feat_watch style_cat',
                 'feat_resizable_cat',
                 'feat_fabric content_cat',
                 'feat_case thickness_cat',
                 'feat_jewelry setting_cat',
                 'feat_jacket length_cat']
feats += feats_catched
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats,model)

In [26]:
X = df[feats].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-57.35701911844486, 4.149174649782326)


Weight,Feature
0.2505 ± 0.0095,brand_cat2
0.1094 ± 0.0077,feat_material_cat
0.0241 ± 0.0017,feat_brand_cat
0.0151 ± 0.0027,feat_fabric content_cat
0.0124 ± 0.0011,feat_weight_cat
0.0106 ± 0.0032,feat_adjustable_cat
0.0091 ± 0.0008,feat_case thickness_cat
0.0055 ± 0.0024,feat_resizable_cat
0.0052 ± 0.0002,feat_gender_cat
0.0045 ± 0.0005,feat_movement_cat


In [27]:
df['brand'].value_counts(normalize=True)

nike                         0.097210
puma                         0.033315
ralph lauren                 0.028775
vans                         0.021116
new balance                  0.020295
                               ...   
zanzara                      0.000055
e2 sport balance             0.000055
ralph lauren purple label    0.000055
us toy                       0.000055
o'neal                       0.000055
Name: brand, Length: 1732, dtype: float64

In [28]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'football', 'condition': 'new with box', 'type': 'cleats'},
       {'shoe size': '12', 'gender': 'men', 'shoe category': "men's shoes", 'brand': 'nike', 'color': 'photo blue/laser orange-deep royal blue'},
       {'season': 'all-season', 'material': 'synthetic', 'gender': 'men', 'shoe size': '11', 'size': '11', 'color': 'black', 'model': '599525 001', 'manufacturer part number': '599525 001', 'brand': 'nike', 'age group': 'adult'},
       {'style': 'running, cross training', 'country/region of manufacture': 'china', 'country of manufacture': 'china', 'condition': 'new with box'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'soccer cleats'}],
      dtype=object)

In [29]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
mens                   4
infant                 4
boys'                  3
men||women             2
women ,�� unisex       2
youth                  2
women                  2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64

In [30]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
!git add matrix_one/day5.ipynb

In [0]:
!git config --global user.email 'janeta.mateusz@gmail.com'
!git config --global user.name 'SirMatix'

In [36]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   matrix_one/day5.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   matrix_one/day3.ipynb
	modified:   matrix_one/day5.ipynb

